# Volumes

Let's get a little further in the use of volumes. We will discover in this reading the different types of volumes that exists and how to manipulate them.

Here is the summary.

* Named volume

* Host volume

* Anonymous volumes

* tmpfs

By default all files created inside a container are stored on a writable container layer.

- The data doesn’t persist when that container no longer exists, and it can be difficult to get the data out of the container if another process needs it.
- A container’s writable layer is tightly coupled to the host machine where the container is running. You can’t easily move the data somewhere else.
- Writing into a container’s writable layer requires a [storage driver](https://docs.docker.com/storage/storagedriver/) to manage the filesystem. The storage driver provides a union filesystem, using the Linux kernel. This extra abstraction reduces performance as compared to using *data volumes*, which write directly to the host filesystem.

## Named volume

Created and managed by Docker. You can create a volume explicitly using the `docker volume create` command, or Docker can create a volume during container or service creation. This is the type of volume we used in this course.
They are useful to keep the data of a container (mysql) and avoid losing useful data when a container is stopped.

 We're going to create our first volume:

In [ ]:
%%bash
docker volume create firstvolume;

You can list the volumes created:

In [ ]:
%%bash
docker volumes ls;
DRIVER              VOLUME NAME
local               firstvolume
local               782e6fcc71ea773dbed31b0588197dedd93b5784dbee92bc4c7a74e541fffaf5

You probably noticed the presence of volume with an incomprehensible name. These are anymes volumes created by the docker daemon to increase the storage space of the containers. We will come back to this in a section below.

You can bind this volume with a container in an interactive shell:

In [ ]:
%%bash
docker container run -v firstvolume:/data -it alpine;

We will create a file in it to see if it persists after the container is stopped.

In [ ]:
%%bash
/ # touch /data/file1

Let's start another container with this volume to see if this file is persisted on the host.

In [ ]:
%%bash
docker container run -v firstvolume:/data -it --name container2 alpine sh

In [ ]:
%%bash
/ # ls /data
file1

# Host volume

These maps a path from the host into the container with a bind mount. They have the short syntax. This volume is useful to share application source code.

Create a file on our host:

In [ ]:
%%bash
touch sharedFile

Share this file with the container:

In [ ]:
%%bash
docker container run -v ${PWD}:/data -it alpine;

In [ ]:
%%bash
ls data;
sharedFile

## Anonymous volume

These only have a path inside the container.  They are in the form `/path/in/container` and docker will create a default named volume with a guid as the name.  If you create a container with the `--rm` flag, anonymous volumes will also be deleted automatically.

You can use this kind of module to prevent a folder from being written to your host when you share a volume with a container.

Let's create two folders :

In [ ]:
%%bash
mkdir -p app/dependencies

Let's share with a container the app folder, but without sharing the dependencies of folder.

In [ ]:
%%bash
docker container run -v ${PWD}/app:/app -v /app/dependencies -it alpine;

Let's create a file in each folder

In [ ]:
%%bash
touch app/index.html;
touch app/dependencies/utils.js
tree app;
.
├── dependencies
│   └── utils
└── index.html

Now let's take a look at the result on our host :

In [ ]:
%%bash
tree app;
.
├── dependencies
└── index.html

This allows not to overwrite on our host files that should not be rewritten as dependencies of a different version.

## tmpfs

A `tmpfs` mount is not persisted on disk, either on the Docker host or within a container. It can be used by a container during the lifetime of the container, to store non-persistent state or sensitive information. For instance, internally, swarm services use `tmpfs` mounts to mount [secrets](https://docs.docker.com/engine/swarm/secrets/) into a service’s containers.

In [ ]:
%%bash
docker run -d --name tmptest --tmpfs /app nginx:latest;

Do not hesitate to go to see the official [Docker Documentation](https://docs.docker.com/storage/).